#Importing required Libraries

In [17]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00


In [18]:
!from transformers import pipeline

/bin/bash: line 1: from: command not found


In [3]:
clf  = pipeline("sentiment-analysis")

clf(["I love playing cricket",
     "I hate basketball"])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9995773434638977},
 {'label': 'NEGATIVE', 'score': 0.9965907335281372}]

#Preparing Input for the model

In [4]:
from transformers import AutoTokenizer

In [5]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
raw_text = ["I love playing cricket",
            "I hate basketball"]

inputs = tokenizer(raw_text, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 1045, 2293, 2652, 4533,  102],
       [ 101, 1045, 5223, 3455,  102,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0]], dtype=int32)>}


In [7]:
from transformers import TFAutoModel

In [8]:
model = TFAutoModel.from_pretrained(checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [10]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 6, 768)


In [11]:
from transformers import TFAutoModelForSequenceClassification

In [12]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs  = model(inputs)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [13]:
print(outputs.logits)

tf.Tensor(
[[-3.765487   4.0029445]
 [ 3.0818796 -2.595973 ]], shape=(2, 2), dtype=float32)


In [14]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.2269722e-04 9.9957734e-01]
 [9.9659073e-01 3.4092339e-03]], shape=(2, 2), dtype=float32)


In [15]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}